In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime

# ---------------------- 1. 数据读取与预处理 ----------------------
filtered_files = [f for f in os.listdir(os.getcwd()) if f.startswith("pet_funeral_beijing_cleaned_20251110_202439.xlsx")]
if not filtered_files:
    print("❌ No filtered files found!")
    exit()
latest_file = max(filtered_files, key=lambda x: os.path.getmtime(x))
df = pd.read_excel(latest_file)
total = len(df)
print(f"✅ Loaded data: {latest_file} (Total: {total} merchants in Beijing)")

# 提取有效价格+统计缺失数据
def extract_valid_price(price_val):
    if pd.isna(price_val) or str(price_val).strip() in ['人均 -', '', 'None']:
        return None
    match = re.search(r'(\d+)', str(price_val))
    return float(match.group()) if match else None

df['valid_price'] = df['avg_price'].apply(extract_valid_price)
valid_data = df[df['valid_price'].notna()]['valid_price']
missing_count = total - len(valid_data)

# ---------------------- 2. 数据准备 ----------------------
price_bins = [0, 500, 800, 1200, 1500, float('inf')]
price_labels = ['0-500 RMB', '501-800 RMB', '801-1200 RMB', '1201-1500 RMB', '>1500 RMB', 'Missing Data']
price_groups = pd.cut(valid_data, bins=price_bins, labels=price_labels[:-1], right=False)
price_counts = price_groups.value_counts().sort_index()
price_counts['Missing Data'] = missing_count

# ---------------------- 3. 绘制柱状图（标题含Beijing） ----------------------
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial', 'Helvetica']
plt.rcParams['axes.unicode_minus'] = False

fig, ax = plt.subplots(figsize=(12, 7))
colors = ['#64b5f6', '#42a5f5', '#2196f3', '#1976d2', '#0d47a1', '#d0d7e3']
bars = ax.bar(price_counts.index, price_counts.values, color=colors, width=0.65,
              edgecolor='white', linewidth=1.5, alpha=0.9)

# 数据标签
for bar in bars:
    height = bar.get_height()
    percentage = (height / total) * 100
    ax.text(bar.get_x() + bar.get_width()/2, height + 0.5,
            f"{int(height)}\n({percentage:.1f}%)",
            ha='center', va='bottom', fontsize=9, fontweight='bold', color='#333')

# 标题明确标注Beijing
ax.set_title('Per Capita Price Distribution of Pet Funeral Merchants in Beijing',
             fontsize=16, fontweight='bold', color='#2c3e50')
ax.set_xlabel('Price Range', fontsize=12, fontweight='medium')
ax.set_ylabel('Number of Merchants', fontsize=12, fontweight='medium')

# 美化
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', labelsize=10, rotation=30)
ax.tick_params(axis='y', labelsize=10)
ax.set_ylim(0, max(price_counts.values) * 1.2)

# ---------------------- 4. 保存 ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(os.getcwd(), f'beijing_price_distribution_{timestamp}.png')
plt.tight_layout()
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()
print(f"📁 Chart saved to: {save_path}")

✅ Loaded data: pet_funeral_beijing_cleaned_20251110_202439.xlsx (Total: 53 merchants in Beijing)
📁 Chart saved to: /Users/syx/Documents/python脚本/beijing_price_distribution_20251114_020353.png


In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime

# ---------------------- 1. 数据读取与预处理 ----------------------
filtered_files = [f for f in os.listdir(os.getcwd()) if f.startswith("pet_funeral_beijing_cleaned_")]
if not filtered_files:
    print("❌ No filtered files found!")
    exit()
latest_file = max(filtered_files, key=lambda x: os.path.getmtime(x))
df = pd.read_excel(latest_file)
total = len(df)
print(f"✅ Loaded data: {latest_file} (Total: {total} merchants in Beijing)")

# 提取有效评分+统计缺失数据
def extract_valid_rating(star_val):
    if pd.isna(star_val) or str(star_val).strip() in ['0.0', '0.0分', '无评分', '', 'None']:
        return None
    match = re.search(r'(\d+\.?\d*)', str(star_val))
    return float(match.group()) if match and float(match.group()) > 0 else None

df['valid_rating'] = df['star'].apply(extract_valid_rating)
valid_data = df[df['valid_rating'].notna()]['valid_rating']
missing_count = total - len(valid_data)

# ---------------------- 2. 数据准备 ----------------------
rating_bins = [0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ['<3.5(Poor)', '3.5-4.0(Fair)', '4.0-4.5(Good)', '4.5-5.0 (Excellent)', 'Missing Data']
rating_groups = pd.cut(valid_data, bins=rating_bins, labels=rating_labels[:-1], right=False)
rating_counts = rating_groups.value_counts().sort_index()
rating_counts['Missing Data'] = missing_count

# ---------------------- 3. 绘制饼图（蓝系色调，与热力图统一） ----------------------
plt.style.use('seaborn-v0_8-white')
plt.rcParams['font.family'] = ['Arial', 'Helvetica']

fig, ax = plt.subplots(figsize=(9, 9))

# 核心调整：蓝系渐变配色（与热力图蓝色系保持一致，从浅到深）
colors = [
    '#b8ddff',  # 浅蓝（对应<3.5）
    '#8ec5fc',  # 中浅蓝（对应3.5-4.0）
    '#5bacf4',  # 中蓝（对应4.0-4.5）
    '#3d85c6',  # 深蓝（对应4.5-5.0）
    '#d0d7e3'   # 浅灰蓝（缺失数据，与蓝系协调）
]
wedgeprops = {'edgecolor': 'white', 'linewidth': 2}  # 白色边框区分扇区

wedges, texts, autotexts = ax.pie(
    rating_counts.values,
    labels=rating_labels,
    colors=colors,
    autopct='%1.1f%%',
    startangle=90,
    pctdistance=0.82,
    labeldistance=1.1,
    wedgeprops=wedgeprops,
    textprops={'fontsize': 11}
)

# 美化标签（与蓝系色调协调）
for autotext in autotexts:
    autotext.set_color('white')  # 百分比文字白色，与蓝色扇区对比清晰
    autotext.set_fontweight('bold')
for text in texts:
    text.set_fontweight('bold')
    text.set_color('#1c4587')  # 标签文字用深蓝，与热力图标题色一致

# 标题（与热力图风格统一）
ax.set_title('Distribution of Pet Funeral Merchant Ratings in Beijing',
             fontsize=16, fontweight='bold', color='#1c4587')  # 标题用深蓝
# 图例（补充数量，保持蓝系协调）
legend_labels = [f'{label} (n={int(rating_counts[i])})' for i, label in enumerate(rating_labels)]
ax.legend(legend_labels, loc='center left', bbox_to_anchor=(1, 0.5),
          fontsize=10, frameon=True, edgecolor='#b8ddff')  # 图例边框用浅蓝

# ---------------------- 4. 保存 ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(os.getcwd(), f'beijing_rating_distribution_{timestamp}.png')
plt.tight_layout()
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()
print(f"📁 Chart saved to: {save_path}")

✅ Loaded data: pet_funeral_beijing_cleaned_20251110_202439.xlsx (Total: 53 merchants in Beijing)


/var/folders/9q/ywtdr2kx78d4gg56pbn1dk_w0000gn/T/ipykernel_23441/3658595038.py:75: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  legend_labels = [f'{label} (n={int(rating_counts[i])})' for i, label in enumerate(rating_labels)]


📁 Chart saved to: /Users/syx/Documents/python脚本/beijing_rating_distribution_20251113_033111.png


In [14]:
# 安装中国及城市地图数据（包含北京）
%pip install echarts-china-cities-pypkg==0.0.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 1.4 MB/s eta 0:00:00a 0:00:010m
  Preparing metadata (setup.py) ... done
  Using cached pyecharts_jupyter_installer-0.0.3-py2.py3-none-any.whl.metadata (2.2 kB)
Using cached pyecharts_jupyter_installer-0.0.3-py2.py3-none-any.whl (4.7 kB)
  DEPRECATION: Building 'echarts-china-cities-pypkg' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'echarts-china-cities-pypkg'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for echarts-china-cities-pypkg: filename=echarts_china_cities_pypkg-0.0.9-py3-none-any.whl size=4272267 sha256=94247fd351406b7ca9146aa87da704bd18f90e538f8a934cfe3e46260cb4829e
  Stored in direc

In [20]:
%pip install pyecharts==1.9.1

  Attempting uninstall: pyecharts
    Found existing installation: pyecharts 2.0.9
    Uninstalling pyecharts-2.0.9:
      Successfully uninstalled pyecharts-2.0.9
Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd
import os
from pyecharts import options as opts
from pyecharts.charts import Map
from datetime import datetime

# ---------------------- 配置路径 ----------------------
INPUT_CLEANED_DATA = "/Users/syx/Documents/python脚本/beijing_second_cleaned_20251113.xlsx"  # 替换为你的清洗文件路径
OUTPUT_HEATMAP = f"北京宠物店铺热力图_{datetime.now().strftime('%Y%m%d')}.html"


# ---------------------- 北京所有行政区列表（确保无遗漏） ----------------------
all_beijing_districts = [
    "东城区", "西城区", "朝阳区", "海淀区", "丰台区", "石景山区",
    "通州区", "顺义区", "昌平区", "大兴区", "房山区", "门头沟区",
    "平谷区", "密云区", "怀柔区", "延庆区"
]


# ---------------------- 生成最终版热力图 ----------------------
if os.path.exists(INPUT_CLEANED_DATA):
    df = pd.read_excel(INPUT_CLEANED_DATA)
    if "district_cn" in df.columns:
        # 统计各行政区店铺数量（无数据的区默认为0）
        district_counts = df["district_cn"].value_counts().to_dict()
        for dist in all_beijing_districts:
            if dist not in district_counts:
                district_counts[dist] = 0
        heatmap_data = [(k, v) for k, v in district_counts.items()]

        # 纯蓝渐变配色
        blue_gradient = [
            "#f0f7ff", "#d9eaff", "#b8ddff", "#8ec5fc",
            "#5bacf4", "#3d85c6", "#2c6aa0", "#1c4587"
        ]

        c = (
            Map(
                init_opts=opts.InitOpts(
                    width="1200px",
                    height="700px",
                    #page_title="北京宠物殡葬商铺热力图"
                    page_title="Beijing Pet Funeral Merchants Regional Distribution Heatmap"
                )
            )
            .add(
                #series_name="店铺数量",
                series_name="Number of Merchants",
                data_pair=heatmap_data,
                maptype="北京",
                label_opts=opts.LabelOpts(
                    is_show=True, 
                    font_size=10,
                    color="#333"
                )
            )
            .set_global_opts(
                title_opts=opts.TitleOpts(
                    #title="北京宠物殡葬店铺区域分布热力图",
                    title="Distribution of Pet Funeral Stores in Beijing",
                    pos_left="center",
                    pos_top="15px",
                    title_textstyle_opts=opts.TextStyleOpts(
                        font_size=18,
                        color="#1c4587",
                        font_weight="bold"
                    )
                ),
                legend_opts=opts.LegendOpts(
                    pos_bottom="10px",
                    pos_left="center",
                    textstyle_opts=opts.TextStyleOpts(color="#3d85c6")
                ),
                visualmap_opts=opts.VisualMapOpts(
                    min_=0,
                    max_=max([v for _, v in heatmap_data]) if heatmap_data else 1,
                    range_color=blue_gradient,
                    pos_right="20px",
                    pos_top="center",
                    orient="vertical",
                    is_calculable=True,
                    textstyle_opts=opts.TextStyleOpts(color="#3d85c6")
                ),
                toolbox_opts=opts.ToolboxOpts(
                    is_show=True,
                    feature={
                        "zoom": {"show": True, "title": "放大"},
                        "restore": {"show": True, "title": "重置"}
                    },
                    pos_right="30px",
                    pos_top="50px",
                    item_size=15
                )
            )
            .set_series_opts(
                # 悬停提示框：区域名在上，店铺数量在下且对齐
                tooltip_opts=opts.TooltipOpts(
                    formatter=lambda params: f"""
                    <div style="text-align:center; min-width:100px;">
                        <div style="font-weight:bold; color:#1c4587;">{params.name}</div>
                        <div style="color:#3d85c6;">{
                            "暂无店铺" if params.value == 0 
                            else f"Number of Merchants：{params.value}"
                        }</div>
                    </div>
                    """,
                    background_color="rgba(220, 235, 255, 0.9)",
                    border_color="#8ec5fc",
                    textstyle_opts=opts.TextStyleOpts(color="#1c4587")
                ),
                itemstyle_opts=opts.ItemStyleOpts(
                    border_color="#fff",
                    border_width=1
                )
            )
        )

        c.render(OUTPUT_HEATMAP)
        print(f"✅ 热力图已保存：{OUTPUT_HEATMAP}")
    else:
        print("❌ 数据中无 'district_cn' 列，请检查清洗步骤")
else:
    print(f"❌ 未找到文件：{INPUT_CLEANED_DATA}")

✅ 热力图已保存：北京宠物店铺热力图_20251113.html
